In [ ]:
import pandas as pd

# I am blind without this
%config InlineBackend.figure_format = 'retina' 

# see all columns
pd.set_option("max_columns", None)
pd.set_option("max_columns", 50)

#--------------------------------- # LOAD DATA # --------------------------------- #
# Case_offense clean dataset
offenses = pd.read_csv(
    "/Users/clarissaache/Documents/Capstone/case_offense_v01.csv", #/Users/clarissaache/Documents/Capstone/case_offense_v01.csv
    low_memory=False,
    index_col=[0],
)
#subset
offs = offenses[
    [
        "case_id",
        "date_of_birth",
        "key_county_num",
        "key_year",
        "court_type",
        "race",
        "sex",
        "process_served",
        "case_creation_date",
        "case_trial_date",
        "court_attorney_type",
        "bond_type",
        "bond_amount",
        "charged_offense_date",
        "charged_offense_code",
        "offense_class",
        # "Offense Description",
        # "CL",
        #'T',
        "min_sentence",
        "convicted_offense_code",
        "disposition",
        "disposition_date",
    ]
]

offs["charge_count"] = 1

# Offense descriptions, classes, and codes
offense_desc_final = pd.read_csv('/Users/clarissaache/Documents/Capstone/JUSTFAIR_capstone/00_raw_data/ACIS_offence_codes_FIXED.csv')

#--------------------------------- # Merge with offence code CONVICTED descriptions: # --------------------------------- #

# rename cols
offense_desc_1 = offense_desc_final.rename(
    columns={
        "Offense Description": "conv_Off_Desc",
        "CODE": "conv_CODE",
        "T": "convc_type",
        "NC General Statute": "conv_statute",
        "CL_min": "conv_CL_min",
        "CL_Rank_min": "conv_CL_Rank_min",
        "CL_max": "conv_CL_max",
        "CL_Rank_max": "conv_CL_Rank_max",
    },
)

# get rid of the ones that dont have convicted code
offs = offs.loc[~offs["convicted_offense_code"].isna()]

# make convictd off code int
offs["convicted_offense_code"] = offs["convicted_offense_code"].astype(int)

# merge
offs_rank = pd.merge(
    left=offs,
    right=offense_desc_1,
    left_on="convicted_offense_code",
    right_on="conv_CODE",
    how="left",
    validate="m:m",
    indicator=True,
)
offs_rank._merge.value_counts()